In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning, FitFailedWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=FitFailedWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [3]:
from pyspark.sql import SparkSession
import pandas as pd

from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .appName("Load Stock Data") \
      .getOrCreate()

23/12/31 23:33:46 WARN Utils: Your hostname, nathan resolves to a loopback address: 127.0.1.1; using 192.168.1.29 instead (on interface enp34s0)
23/12/31 23:33:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/31 23:33:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/31 23:33:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
np.random.seed(42)

In [5]:
df = spark.read.csv("./train/train.csv", header=True, inferSchema=True)
df.show()

23/12/31 23:34:19 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+------------------+----------+--------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+------------------+------------------+------------------+------------------+--------------------+--------------+-----------+---------------+------------------+---------------+------------------+---------------+------------------+----------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------------+--------------------+-------------------------+-------------------------+-----------------+--------------------+--------------------+------+
|exchange|               RSI|RSI_signal|                 ATR|      EMA_relative|       MA_relative|        SMA_relative|        WMA_relative|                MACD|       MACD_relative|MACD_relative_signal|MACD_relative_histogram|               ADX|   Stoch_O

Our Data has no Null Values!

In [6]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.feature import StringIndexer, VectorAssembler

categorical_columns = ["exchange", "RSI_signal", "Stoch_O_signal", "fibonacci_signal", "Bollinger_signal", "ichimoku_c_signal"]
numerical_feature_columns = [
    "RSI", "ATR", "EMA_relative", "MA_relative", "SMA_relative", 
    "WMA_relative", "MACD", "MACD_relative", "MACD_relative_signal", 
    "MACD_relative_histogram", "ADX", "Stoch_O_k_value", "Stoch_O_k_smoothed", 
    "Stoch_O_d_value", "pivot_point", "pivot_support_1", "pivot_resistance_1", 
    "pivot_support_2", "pivot_resistance_2", "pivot_support_3", 
    "pivot_resistance_3", "VWAP_relative_short", "VWAP_relative_long", 
    "Bollinger_upper_band", "Bollinger_lower_band", "ichimoku_c_conversion_line", 
    "ichimoku_c_base_line", "ichimoku_c_leading_span_a", "ichimoku_c_leading_span_b", 
    "SAR_relative", "s&p500_move_15m"
]

indexers = [StringIndexer(inputCol=c, outputCol=c+"_index").fit(df) for c in categorical_columns]
indexed_df = df
for indexer in indexers:
    indexed_df = indexer.transform(indexed_df)

for c in categorical_columns:
    indexed_df = indexed_df.drop(c)

indexed_cols = [c + "_index" for c in categorical_columns]
feature_columns = indexed_cols + numerical_feature_columns
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
final_df = assembler.transform(indexed_df)

final_df.show()

+------------------+--------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+------------------+------------------+------------------+------------------+--------------------+-----------+---------------+------------------+---------------+------------------+---------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------------+--------------------+-------------------------+-------------------------+--------------------+--------------------+------+--------------+----------------+--------------------+----------------------+----------------------+-----------------------+--------------------+
|               RSI|                 ATR|      EMA_relative|       MA_relative|        SMA_relative|        WMA_relative|                MACD|       MACD_relative|MACD_relative_signal|MACD_relative_hi

splitting the dataset into train and test

In [7]:
#80-20 train test split
train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=42)
train_df.show()

+---+--------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+------------------+-----------------+------------------+------------------+--------------------+-----------+---------------+------------------+---------------+------------------+---------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------------+--------------------+-------------------------+-------------------------+--------------------+--------------------+------+--------------+----------------+--------------------+----------------------+----------------------+-----------------------+--------------------+
|RSI|                 ATR|      EMA_relative|       MA_relative|        SMA_relative|        WMA_relative|                MACD|       MACD_relative|MACD_relative_signal|MACD_relative_histogram|               ADX|  St

perform hyper-parameter tuning via cross validation to find optimum parameters for random forrest classifier

In [8]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol='features', labelCol='result')
paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [10, 20])
             .addGrid(rf.maxDepth, [5, 10])
             .addGrid(rf.impurity, ["entropy"])
             .build())

evaluator = MulticlassClassificationEvaluator(labelCol='result', predictionCol='prediction', metricName='accuracy')
crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

cvModel = crossval.fit(train_df)

23/12/31 23:35:17 WARN MemoryStore: Not enough space to cache rdd_68_0 in memory! (computed 67.7 MiB so far)
23/12/31 23:35:17 WARN BlockManager: Persisting block rdd_68_0 to disk instead.
23/12/31 23:35:18 WARN MemoryStore: Not enough space to cache rdd_68_1 in memory! (computed 67.8 MiB so far)
23/12/31 23:35:18 WARN BlockManager: Persisting block rdd_68_1 to disk instead.
23/12/31 23:35:26 WARN MemoryStore: Not enough space to cache rdd_68_2 in memory! (computed 67.7 MiB so far)
23/12/31 23:35:26 WARN BlockManager: Persisting block rdd_68_2 to disk instead.
23/12/31 23:35:33 WARN MemoryStore: Not enough space to cache rdd_68_5 in memory! (computed 67.7 MiB so far)
23/12/31 23:35:33 WARN BlockManager: Persisting block rdd_68_5 to disk instead.
23/12/31 23:36:00 WARN MemoryStore: Not enough space to cache rdd_68_1 in memory! (computed 67.8 MiB so far)
23/12/31 23:36:01 WARN MemoryStore: Not enough space to cache rdd_68_3 in memory! (computed 67.7 MiB so far)
23/12/31 23:36:01 WARN Mem

Testing

In [9]:
predictions = cvModel.transform(test_df)
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g" % accuracy)

23/12/31 23:57:37 WARN DAGScheduler: Broadcasting large task binary with size 1860.5 KiB


Test Accuracy = 0.919725
